Forms k clusters (k found using inertia plots, choosing the elbow point), and labels cluster as positive or negative based on the majority of tweets in the cluster (KMeans)

In [1]:
import pandas as pd 
from scipy import spatial 
import gensim.models
import json
import numpy as np 
from sklearn.cluster import KMeans

C:\Users\Aman Garg\AppData\Roaming\Python\Python27\site-packages\gensim-2.1.0-py2.7-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"


In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [3]:
ff=open('C:\Users\Aman Garg\Desktop\Project\Codes\phrases_final_updated.json','r') #Opening json file containing phrases extracted from tweets
ph=json.load(ff)
so={}
solist=[]
phr={}

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:\Users\Aman Garg\Documents\GoogleNewsDataset\GoogleNews-vectors-negative300.bin', binary=True)


In [46]:
not_found_counter = 0 # Counter for phrases not found in word2vec
tweet_vec_list = []
for j in range(8000):
	k=unicode(j)
	phr[k]=ph[k] #phr[k] - list of phrases for tweet number k
	phrase_vec_list = []
	for i in phr[k]:
		#print i
		phrase_words = i.split()
		#print phrase_words
		try:
			word_sum = (model[phrase_words[0]]+ model[phrase_words[1]])/2
			phrase_vec_list.append(word_sum)
		except:
			not_found_counter = not_found_counter + 1
			pass
	try:
		tweet_vec = sum(phrase_vec_list)/len(phrase_vec_list)
	except:
		tweet_vec = [0]*300
	tweet_vec_list.append(tweet_vec)
	#print tweet_vec

In [144]:
samples = np.array(tweet_vec_list)
new_samples = samples[:]
samples = samples[:]

In [ ]:
#Code for inertia plot to find elbow point 

# ks = range(1,40)
# Inertias = []
# for k in ks:
#     kmeans = KMeans(n_clusters = k)
#     labels = kmeans.fit(samples)
#     Inertias.append(kmeans.inertia_)
# # Plot ks vs inertias
# plt.plot(ks, Inertias, '-o')
# plt.xlabel('number of clusters, k')
# plt.ylabel('inertia')
# plt.xticks(ks)
# plt.show()


Choose optimal k (elbow point)

After Standardization

In [169]:
# Put opt k
k_opt = 20
kmeans = KMeans(n_clusters = k_opt)

In [170]:
scaler = StandardScaler()
pipeline = make_pipeline(scaler,kmeans)
pipeline.fit(samples)
y_pred = pipeline.predict(new_samples)

In [171]:
print len(y_pred)

8000


In [172]:
df2 = pd.read_csv('C:\Users\Aman Garg\Desktop\Project\Codes\word2vec_with_dict_accuracy.csv')

In [173]:
y_test =df2['actual'].values
y_test = y_test[:]
#print len(y_test)
#print len(y_pred)
for i in range(k_opt):    
    print "cluster "+str(i)+ " count:"+str(sum(y_pred == i))
    a = sum(y_test[y_pred == i] == 'positive')
    print "a:" + str(a)
    b = sum(y_test[y_pred == i] == 'negative')
    print "b:"+ str(b)
    print "max: " + str(np.argmax([a,b]))
    if max(a,b) == 0:
        print "No tweets of this cluster"
    else:
        print "Accuracy:" + str(((max(a,b)+0.0)/(a+b))*100)
    #if np.argmax([a,b]) == 0: #if a>b
    #    y_pred[y_pred==i] = 1
    #else:
    #    y_pred[y_pred==i] = 0

cluster 0 count:323
a:242
b:81
max: 0
Accuracy:74.9226006192
cluster 1 count:1578
a:1162
b:416
max: 0
Accuracy:73.6375158428
cluster 2 count:584
a:210
b:374
max: 1
Accuracy:64.0410958904
cluster 3 count:114
a:105
b:9
max: 0
Accuracy:92.1052631579
cluster 4 count:143
a:112
b:31
max: 0
Accuracy:78.3216783217
cluster 5 count:180
a:146
b:34
max: 0
Accuracy:81.1111111111
cluster 6 count:119
a:63
b:56
max: 0
Accuracy:52.9411764706
cluster 7 count:261
a:217
b:44
max: 0
Accuracy:83.1417624521
cluster 8 count:601
a:429
b:172
max: 0
Accuracy:71.381031614
cluster 9 count:803
a:596
b:207
max: 0
Accuracy:74.2216687422
cluster 10 count:126
a:96
b:30
max: 0
Accuracy:76.1904761905
cluster 11 count:506
a:380
b:126
max: 0
Accuracy:75.0988142292
cluster 12 count:371
a:346
b:25
max: 0
Accuracy:93.2614555256
cluster 13 count:270
a:227
b:43
max: 0
Accuracy:84.0740740741
cluster 14 count:225
a:161
b:64
max: 0
Accuracy:71.5555555556
cluster 15 count:896
a:559
b:337
max: 0
Accuracy:62.3883928571
cluster 16 cou

In [174]:
for j in range(k_opt):
    a = sum(y_test[y_pred == j] == 'positive')
    b = sum(y_test[y_pred == j] == 'negative')
    print "positives: " + str(a)
    print "negatives: " + str(b)
    if a>b: #More positive labels in cluster than negative
        print "Assigning positive to cluster:" + str(j)
        for index,k in enumerate(y_pred):
            if k ==j:
                y_pred[index] = 1
    else:
        print "Assigning negative to cluster:" + str(j)
        for index,k in enumerate(y_pred):
            if k==j:
                y_pred[index] = 0

positives: 242
negatives: 81
Assigning positive to cluster:0
positives: 1404
negatives: 497
Assigning positive to cluster:1
positives: 210
negatives: 374
Assigning negative to cluster:2
positives: 105
negatives: 9
Assigning positive to cluster:3
positives: 112
negatives: 31
Assigning positive to cluster:4
positives: 146
negatives: 34
Assigning positive to cluster:5
positives: 63
negatives: 56
Assigning positive to cluster:6
positives: 217
negatives: 44
Assigning positive to cluster:7
positives: 429
negatives: 172
Assigning positive to cluster:8
positives: 596
negatives: 207
Assigning positive to cluster:9
positives: 96
negatives: 30
Assigning positive to cluster:10
positives: 380
negatives: 126
Assigning positive to cluster:11
positives: 346
negatives: 25
Assigning positive to cluster:12
positives: 227
negatives: 43
Assigning positive to cluster:13
positives: 161
negatives: 64
Assigning positive to cluster:14
positives: 559
negatives: 337
Assigning positive to cluster:15
positives: 181

In [175]:
count = 0
pos_pred_count = 0
neg_pred_count = 0
for index,value in enumerate(y_pred):
    if y_test[index] == 'positive' and value == 1:
        count = count +1
    elif y_test[index] == 'negative' and value == 0:
        count = count + 1
    if value == 1:
        pos_pred_count = pos_pred_count + 1
    else:
        neg_pred_count = neg_pred_count + 1
print 'Accuracy: %d/8000' %count
print 'Positive predictions :' + str(pos_pred_count)
print 'Negative predictions :' + str(neg_pred_count)
# for k = 100, with training -> 6000 and test -> 2000 : Accuracy 1510/2000
# for k = 20, with training -> 6000 and test -> 2000 : Accuracy 1478/2000
# for k = 2, with training -> 6000 and test -> 2000 : Accuracy 1478/2000
# for k = 40, with training -> 6000 and test -> 2000 : Accuracy 1499/2000

#for k = 40, with training and testing 8000 (same set) : Accuracy 5820/8000
#for k = 2, with training and testing of 8000 (same set) : Accuracy 5649/8000
#for k = 20, with training and testing of 8000 (same set) : Accuracy 5813/8000

Accuracy: 5813/8000
Positive predictions :7416
Negative predictions :584
